# Generate daily VWAP using dedicated service - Python

### Overview
The Volume Weighted Average price is similar to a moving average, except volume is included to weight the average price over a one trading day period.

It serves as a reference point for one trading day prices. Comparing current prices with the VWAP values enables to:
* mesure trading efficiency by comparing individual trades to VWAP
* highlight liquidity points as vwap reflects prices levels weighted by volume
* determine the intraday trend
* etc.

This sample demonstrates how to request and plot **on-demand** VWAP price from a dedicated `daily data store`.

### Inputs/outputs
Daily VWAP indicator requires instrument's identifier, date time intervals and time range as per inputs and returns the daily VWAP data points.

VWAP is a cumulative indicator which means the number of data points progressively increases throughtout the day.

This sample shows how to plot a simple daily vwap for basis technical analysis using an open source library. 

### Services used
This sample uses gRPC requests in order to retrieve VWAP indicator from the hosted service. The queried endpoint in this script are:
* *DailyVwapsService*: to directly retrieve daily VWAP objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.type.shared.v1*
    * *systemathics.apis.services.daily.v1*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* as per display package

***

# Run daily VWAP sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.daily.v1.daily_vwap_pb2 as daily_vwap
import systemathics.apis.services.daily.v1.daily_vwap_pb2_grpc as daily_vwap_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily vwap* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyVwapsService* in order to retrieve daily data

In [ ]:
# set the instrument
ticker = 'ACA'
exchange = 'XPAR'

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
# create the daily vwap request
request = daily_vwap.DailyVwapsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the daily vwaps service
    service = daily_vwap_service.DailyVwapsServiceStub(channel)
    
    # process the daily vwaps request
    response = service.DailyVwaps(
        request = request, 
        metadata = [('authorization', token)]
    )
    
print("Total daily vwaps retrieved: ",len(response.data))

### Step 4: Visualize data

#### 4.1 Retrieve daily prices
In the following code snippets, the reply is reprocessed in a dataframe in order to visualize the results:

In [ ]:
# prepare the dataframe content
dates=[datetime(v.date.year, v.date.month, v.date.day) for v in response.data]
vwaps = [v.price for v in response.data]
volumes = [v.volume for v in response.data]

d = {'Date': dates, 'Vwap': vwaps, 'Volume': volumes }

In [ ]:
# create pandas dataframe
df = pd.DataFrame(data=d)

In [ ]:
# visualize dataframe
df

#### 4.2 Plot VWAP
The following code snippets display daily VWAP and volume request results with the package of your choice.<br>*`plotly`* is used as per open source display package.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['Date'], y=df['Vwap'], name='Vwap'), secondary_y=True)
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], name="Volume"), secondary_y=False)

fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'VWAP and volume for {0}-{1}'.format(ticker,exchange))
fig.layout.yaxis2.showgrid=False
fig.show()